In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


Configurar  TASK

In [0]:
TASK = 'QNLI'
splits = [
 f'{TASK}/train.tsv', 
 f'{TASK}/test.tsv',
 f'{TASK}/dev.tsv',
]
label_idx = [
             'question',
             'sentence',
            ]

Baixar TASK

In [3]:
!wget https://raw.githubusercontent.com/nyu-mll/GLUE-baselines/master/download_glue_data.py
!python download_glue_data.py -d . -t $TASK

--2020-06-05 17:55:13--  https://raw.githubusercontent.com/nyu-mll/GLUE-baselines/master/download_glue_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8004 (7.8K) [text/plain]
Saving to: ‘download_glue_data.py.1’

download_glue_data. 100%[===================>]   7.82K  --.-KB/s    in 0.001s  

2020-06-05 17:55:13 (5.48 MB/s) - ‘download_glue_data.py.1’ saved [8004/8004]

	Completed!


Translate

Prepare input

In [3]:
!pip install transformers --upgrade
!pip install mosestokenizer

# Translate sentences to English and save mapping dicitionary as json.
# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

import json
import math
import os
import torch

from tqdm.notebook import tqdm
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

def decode(text):
    decoded = tokenizer.decode(
                    text, 
                    skip_special_tokens=True,
                    )

    return decoded

def needs_translation(sample, translations):
    needs = [text for text in sample if text not in translations.keys()]
    
    return needs

def translation(sample):
    src_txts = [f'>>pt_BR<< {text}' for text in sample]

    #translation_sample = tokenizer.prepare_translation_batch(src_txts)
    #translated = model.generate(translation_sample)

    model_inputs = tokenizer.prepare_translation_batch(src_txts).to("cuda")
    generated_ids = model.generate(
                                    model_inputs.input_ids, 
                                    attention_mask=model_inputs.attention_mask, 
                                    num_beams=2
                                    )
    
    translated = [decode(text) for text in generated_ids]

    return translated


def translate2dict(sentences, dictpath, batch_size):
    if not os.path.isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    with open(dictpath) as f:
        translations = json.load(f)
    
    remaining = needs_translation(sentences, translations)
    batch = math.ceil(len(remaining)/batch_size)

    for idx in tqdm(range(batch), "Translating"):
        keys = remaining[idx*batch_size:(idx+1)*batch_size] 
        values = translation(keys)
    
        new_translations = dict(zip(keys, values))
        translations.update(new_translations)
        torch.cuda.empty_cache()
        with open(dictpath, 'w+') as f:
            json.dump(translations, f)

from pprint import pprint
import pandas as pd
import os

sentences = list()
length = 0

for f in splits:
    table = pd.read_csv(f, sep = '\t', quoting=3, error_bad_lines=False)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size
        


assert length == len(sentences)
sentences = set(sentences)

     |████████████████████████████████| 675kB 4.6MB/s 
     |████████████████████████████████| 890kB 19.6MB/s 
     |████████████████████████████████| 1.1MB 10.9MB/s 
     |████████████████████████████████| 3.8MB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e4e53f28ca1dc46c652e9ee31e9506eb694cb3e3cb9d676d43e7f56d93f37bf9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-cp36-none-any.whl size=49120 sha256=0f776e6ae0589450c762345b0881dd7db0c71749550354748c6332140aa44d4c
  Stored in directory: /root/.cache/pip/wheels/a2/e7/48/48d5e0f9c0cd5def2dfd7cb8543945f906448ed1313de24a29
  Created wheel for uctools: filename=uctools-1.2.1-cp36-none-any.whl size=3992 sha256=9ee0b5394132d6630921e369d4115936eb2f145098f639d0af3cb27ad275a60c
  Stored in directory: /root/.cache/pip/wheels/f3/97/

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,



QNLI/train.tsv
0                         What is the Grotto at Notre Dame?
1                         What is the Grotto at Notre Dame?
2         What sits on top of the Main Building at Notre...
3         What sits on top of the Main Building at Notre...
4         When did the Scholastic Magazine of Notre dame...
                                ...                        
108431    What year did the OKL become aware that Britai...
108432    Why did samurai avoid divorcing for reasons of...
108433    Why did samurai avoid divorcing for reasons of...
108434    What type of people did Kanye state had more p...
108435    What type of people did Kanye state had more p...
Name: question, Length: 108436, dtype: object



0         Immediately behind the basilica is the Grotto,...
1         It is a replica of the grotto at Lourdes, Fran...
2         Atop the Main Building's gold dome is a golden...
3         Next to the Main Building is the Basilica of t...
4         Begun as a one-page journ

In [4]:
!nvidia-smi

Fri Jun  5 18:00:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |   1099MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
dictpath = f'{TASK}.json'
translate2dict(list(sentences),  dictpath, 100)

Replace with translation

In [0]:
from os import makedirs
import json

with open(dictpath) as reader:
    dictionary = json.load(reader)

translation_folder = f'translation/{TASK}'
makedirs(translation_folder)
for f in splits:
    table = pd.read_csv(f, sep = '\t', quoting=3, error_bad_lines=False)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, sep = '\t')